In [1]:
#AlexNet Implementation in Keras

In [2]:
#Importing Libraries
import tensorflow.keras as keras
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd 
import os
import sys
import time


In [3]:
folder_path = os.getcwd()
print(folder_path)
checkpoint_path = "\\".join((folder_path,"Training","cp1.ckpt"))

i:\WPI\Fall 2020\Computer Vision - RBE 549\CV Project\AlexNet_Implementation


In [4]:
import pickle 

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
#The data has been saved as a pickle file. 
# Numpy array, Each entry has two rows - first entry - label, the next entry is the image!

In [7]:
default_model_name = 'keras_alexnet.h5'
default_model_dir = 'models'

In [8]:
def AlexNet_Gray_Build(image_height = 256, image_width = 256, image_channel=1, class_count = 2):
    """
    The Original Paper has 224 x 224 x 3. 
    We chose 256 x 256 since it matches with the feature descriptors we have created.
    """
    #To ensure that the shapes and channels are as expected
    #assert  
    
    model = Sequential()

    #Layer 1 - Filters the 224 x 224 x 4 Input Image 
    #           with 96 Kernels of size 11 x 11 x 3
    #           with stride of 4 pixels

    model.add(Conv2D(   filters = 96, 
                        kernel_size = (11, 11),
                        strides = 4,
                        input_shape = (image_height, image_width, image_channel),
                        activation = 'relu',
                        padding = "same"))
    
    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = (2, 2)))
                    
    #Layer 2: 256 Kernels of Size 5 x 5  x 48
    model.add(Conv2D(   filters = 256,
                        kernel_size = (5, 5),
                        activation = 'relu',
                        padding = 'same'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = (2, 2)))
    
    #Layer 3: "384 Kernels of size 3 x 3 x 256" 
    # 256 Represents the prior layer output!
    # How was this arrived? Just random experiments?
    # Same Padding = Zero Padding! Check Stackoverflow!
    model.add(Conv2D(   filters = 384,
                        kernel_size = (3, 3),
                        activation = 'relu',
                        padding = 'same'))

    #Adding these layers in case, we wish to add them.
    #Layer 4: "384 Kernels of size 3 x 3 x 192"
    model.add(Conv2D(   filters = 384,
                        kernel_size = (3, 3),
                        activation = 'relu',
                        padding = 'same'))

    #Layer5: "256 Kernel of size 3 x 3 x 192"
    model.add(Conv2D(   filters = 256,
                        kernel_size = (3, 3),
                        activation = 'relu',
                        padding = 'same'))

    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = (2, 2)))

    model.add(Flatten())

    #Fully Connected with 4096 Neurons Each!
    #Layer 6
    model.add(Dense(units = 4096))
    model.add(Dropout(0.5))   
    #Layer 7
    model.add(Dense(units = 4096))
    model.add(Dropout(0.5))   

    #Prediction Layer - Layer 8
    model.add(Dense(units = class_count))

    #Output Layer - Softmax
    model.add(Activation('softmax'))

    return(model)

In [9]:
#Normalizing Pixel Data!
def norm_pixel(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    #Normalizing Data!
    train_norm = train_norm /255.0
    test_norm = test_norm/255.0

    return train_norm, test_norm

In [10]:
# def load_sample_data():
#     """
#     Load dataset from pickle file and split features and labels
#     returns (X_train, X_test, y_train, y_test)
#     """

#     # start_time = time.time()
#     # with open("sample_train_data.pkl", 'rb') as f_name:
#     #     train_data = pickle.load(f_name)
#     # print("Train Data Shape: ", np.shape(train_data))
#     # assert (np.shape(train_data) == (180, 2)), "Train Shape doesn't match Expectation (180, 2)"
    
#     # with open("sample_test_data.pkl", 'rb') as f_name:
#     #     test_data = pickle.load(f_name)
#     # print("Test Data Shape: ", np.shape(test_data))
#     # assert (np.shape(test_data) == (20, 2)), "Test Shape doesn't match Expectation (20, 2)"
    
#     # X_train = train_data[:, 1]
#     # X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
    
#     # y_train = train_data[:, 0]

#     # X_test = test_data[:, 1]
#     # X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

#     # y_test = test_data[:, 0]
#     # print("Out!")
#     #return(X_train, X_test, y_train, y_test)


In [11]:
def load_shuffled_data():
    """
    Load dataset from pickle file and split features and labels
    returns (X_train, X_test, y_train, y_test)
    """
    start_time = time.time()

    with open("sample_train_img.pkl", 'rb') as f_name:
        train_img = pickle.load(f_name)
        train_img /= 255.0

    with open("sample_train_label.pkl", 'rb') as f_name:
        train_label = pickle.load(f_name)
        train_label = tf.keras.utils.to_categorical(train_label.astype('int'))
        
    
    with open("sample_test_img.pkl", 'rb') as f_name:
        test_img = pickle.load(f_name)
        test_img /= 255.0

    with open("sample_test_label.pkl", 'rb') as f_name:
        test_label = pickle.load(f_name)
        test_label = tf.keras.utils.to_categorical(test_label.astype('int'))

    # img_gen = ImageDataGenerator(rescale = 1.0/255.0)
    # train_iterator = 

    permute_train = np.random.permutation(len(train_label))
    permute_test = np.random.permutation(len(test_label))

    train_img = train_img[permute_train]
    train_label = train_label[permute_train]

    test_img = train_img[permute_train]
    test_label = train_label[permute_train]

    return(train_img, test_img, train_label, test_label)

    

In [12]:
#Don't need this!
# def batch_generator(x_data, y_data, batch_size = 10, class_count = 2, image_height = 256, image_width = 256):
#     """
#     Generate Batch Training.
#     Yield is used when we don't want our generator to forget the index number it used!
#     """

#     X = []
#     Y = []
#     for index in range(0, len(x_data)):
#         X.append(x_data[index])
#         Y.append(y_data[index])
        
#         #Once the batch size is reached:
#         if (index+1) % batch_size == 0:
#             yield(X, Y)
#             X = []
#             Y = []

In [13]:
train_x, _, train_y, _ = load_shuffled_data()
print(train_x[0].shape)


(256, 256, 1)


array([1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1])

In [14]:
#Training the Model:
def train_AlexNet(model, class_count = 2, epochs = 5, batch_size = 10):
    """

    """
    print("Model Summary: \n", model.summary)

    #Compile Model
    model.compile(  loss = 'categorical_crossentropy',
                    optimizer = SGD(lr = 0.001, 
                    momentum=0.9, decay = 0.0005), 
                    metrics = ['accuracy'])
    
    print("Model Compiled!")

    #Training!
    X_train, X_valid, Y_train, Y_valid = load_shuffled_data()
    batch_size = 30
    # steps = len(X_train) / batch_size

    # Train model using Batch Generator!
    # batch = batch_generator(x_data = X_train, y_data = Y_train, class_count = class_count, batch_size = batch_size )

    Early_Stop = keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
    Save_CP = keras.callbacks.ModelCheckpoint(os.getcwd())
    model.fit(  x = X_train,
                y = Y_train,
                epochs = epochs,
                verbose = 1)
    
    print("Training Done!")
    #No need to return

In [15]:
def evaluate_AlexNet(model, x_valid, y_valid, batch_size = 10):
    scores = model.evaluate(x= x_valid, y = y_valid)
    print("Loss Value: ", scores[0])
    print("Metrics: ", scores[1])

In [16]:
ANet_Gray = AlexNet_Gray_Build()
print(ANet_Gray.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 96)        11712     
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 31, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 384)       8

In [17]:
train_AlexNet(ANet_Gray, class_count = 2, epochs = 7)


Model Summary: 
 <bound method Network.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002AF08FF7A48>>
Model Compiled!
Train on 180 samples
Epoch 1/7
180/180 [==============================] - 13s 72ms/sample - loss: 4.5529 - accuracy: 0.4500
Epoch 2/7
160/180 [=========================>....] - ETA: 1s - loss: 1.4518 - accuracy: 0.5156

KeyboardInterrupt: 

In [18]:
evaluate_AlexNet(ANet_Gray)

TypeError: evaluate_AlexNet() missing 2 required positional arguments: 'x_valid' and 'y_valid'

In [34]:
ANet_Gray.save(os.getcwd())
print("Model Saved")

NameError: name 'ANet_Gray' is not defined